In [1]:
import psycopg2
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('postgresql+psycopg2://ml3:1234@db:5432/db_transcendence')

In [4]:
query = "SELECT id, x_ball, y_ball, vel_x_ball, vel_y_ball, pos_hit FROM player_trainingdata"
df = pd.read_sql_query(query, engine)

In [4]:
df = df[(df['pos_hit'] >= 0) | (df['x_ball'] >= 0)]

In [5]:
df.count()

id            110
x_ball        110
y_ball        110
vel_x_ball    110
vel_y_ball    110
pos_hit       110
dtype: int64

In [6]:
df.x_ball = df.x_ball / 10
df.y_ball =  df.y_ball / 10
df.pos_hit = df.pos_hit / 10

In [7]:
df.loc[:100,["x_ball", "y_ball", "vel_x_ball", "vel_y_ball", "pos_hit"]]

,x_ball,y_ball,vel_x_ball,vel_y_ball,pos_hit
0,36.000000,18.000000,2.000000,-2.000000,17.800000
1,48.800000,5.200000,2.000000,-2.000000,17.800000
2,61.600000,9.200000,2.000000,2.000000,17.800000
3,57.844854,20.271915,-5.883403,1.177102,31.454388
4,20.191075,27.805370,-5.883403,1.177102,31.454388
...,...,...,...,...,...
96,20.081692,16.355557,-7.980882,0.552739,17.626857
97,33.245060,26.702519,7.687658,2.213576,33.343247
98,59.176092,30.045325,-6.856078,-4.122402,5.723154
99,15.297194,3.661953,-6.856078,-4.122402,5.723154


In [8]:
pd.set_option('display.max_rows', None)

In [9]:
import tensorflow as ft
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

2024-08-21 14:59:25.245278: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 14:59:25.246327: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 14:59:25.248750: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 14:59:25.254404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 14:59:25.263555: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [10]:
from tensorflow.keras.losses import MeanSquaredError

In [11]:
model = Sequential([
    Dense(units=200, activation='relu'),
    Dense(units=200, activation='relu'),
    Dense(units=1, activation='linear'),
])

In [12]:
model.compile(loss=MeanSquaredError(), optimizer=ft.keras.optimizers.Adam(0.001),)

In [13]:
X = df.loc[:2000,["x_ball", "y_ball", "vel_x_ball", "vel_y_ball"]]
Y = df.loc[:2000,["pos_hit"]]

In [14]:
model.fit(X, Y, epochs=100)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 437.6740  
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 127.6447 
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 123.0127 
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - loss: 116.6312
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - loss: 92.4023
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 83.8297 
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 81.6232 
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 80.2752 
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 74.9312 
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 74.0622 
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 75.8306 
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 75.4608 
Epoch 13/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 71.3980 
Epoch 14/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 75.4119 
Epoch 15/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 74.936

In [15]:
(model.predict(df.loc[1400:1413,["x_ball", "y_ball", "vel_x_ball", "vel_y_ball"]])) * 10

2024-08-21 15:00:20.736488: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/opt/conda/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


ValueError: math domain error

In [16]:
# model.save("model.keras")